In [1]:
pwd

'c:\\Users\\Dell\\Desktop\\Chicken\\Chicken-Disease-Classification\\research'

In [2]:
import os
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Dell\\Desktop\\Chicken\\Chicken-Disease-Classification'

In [5]:
import tensorflow as tf


In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    params_image_size : list
    params_batch_size : int


In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> ModelEvaluationConfig:
        eval_config = ModelEvaluationConfig(
                path_of_model= "artifacts/training/model.h5",
                training_data = "artifacts/data_ingestion/Chicken-fecal-images",
                all_params = self.params,
                params_image_size = self.params.IMAGE_SIZE,
                params_batch_size = self.params.BATCH_SIZE
        )

        return eval_config
    

In [15]:
from urllib.parse import urlparse

In [16]:
class ModelEvaluation:
    def __init__(self, config : ModelEvaluationConfig):
        self.config = config

    def valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation= 'bilinear'
        )

        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs            
        )

    @staticmethod
    def load_model(path : Path) -> tf.keras.Model:
        return tf.keras.models.load_model(str(path))
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_scores(self):
        scores = {"loss" : self.score[0], "accuracy" : self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [17]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = ModelEvaluation(val_config)
    evaluation.evaluation()
    evaluation.save_scores()
except Exception as e:
    raise e

[2023-07-17 22:33:22,612: INFO: common: yaml file : c:\Users\Dell\Desktop\Chicken\Chicken-Disease-Classification\config\config.yaml is successfully added:]
[2023-07-17 22:33:22,619: INFO: common: yaml file : c:\Users\Dell\Desktop\Chicken\Chicken-Disease-Classification\params.yaml is successfully added:]
['artifacts']
[2023-07-17 22:33:22,622: INFO: common: directory : artifacts is successfully created:]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 17s 2s/step - loss: 0.5196 - accuracy: 0.8707
[2023-07-17 22:33:40,019: INFO: common: json file saved at scores.json:]
